In [27]:
import nltk as nltk
import nltk.corpus  
from nltk.text import Text
import pandas as pd
import re
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mohammedhussain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Step 1: extracting regulation descriptions from each record corresponding to a failed inspection

In [42]:
df = pd.read_csv("Food_Inspections.csv")

In [43]:
df.shape

(211348, 17)

In [44]:
df = df.loc[df['Results'] == 'Fail']

In [45]:
df.shape

(40848, 17)

In [33]:
df.head(10)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
6,2373427,THE BUNKER,THE BUNKER,2718517.0,Restaurant,Risk 3 (Low),1515 W MONROE ST,CHICAGO,IL,60607.0,06/09/2020,License,Fail,NaN,41.880086,-87.665054,"(-87.66505441566734, 41.88008566659281)"
15,2369409,DSD DELI,DSD DELI,2626186.0,DELI/GROCERY,Risk 3 (Low),5205 N MILWAUKEE,CHICAGO,IL,60630.0,04/17/2020,License,Fail,NaN,41.974849,-87.766829,"(-87.76682905043768, 41.97484853157327)"
33,2360021,I LOVE SUSHI,I LOVE SUSHI,2594986.0,Restaurant,Risk 1 (High),233 N MICHIGAN AVE,CHICAGO,IL,60601.0,02/06/2020,Canvass,Fail,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.886567,-87.624385,"(-87.62438467059714, 41.886567370886944)"
66,2356510,HAILEY'S HOAGIES,HAILEY'S HOAGIES,2583295.0,Restaurant,Risk 1 (High),1055 W 63RD ST,CHICAGO,IL,60621.0,01/10/2020,Canvass,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.779537,-87.651917,"(-87.65191733069446, 41.779536944269374)"
69,2356448,ATRIUM EVENTS,ATRIUM EVENTS,2684493.0,Restaurant,Risk 1 (High),4325-4343 N RAVENSWOOD AVE,CHICAGO,IL,60613.0,01/09/2020,License,Fail,NaN,41.960476,-87.673799,"(-87.67379931767802, 41.96047630495004)"
70,2356434,SIMPLY THALIA,SIMPLY THALIA,2059952.0,Restaurant,Risk 1 (High),108 N STATE ST,CHICAGO,IL,60602.0,01/09/2020,Canvass,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.883423,-87.628022,"(-87.62802165207536, 41.88342263701488)"
71,2356436,UNCLE REMUS,UNCLE REMUS PLUS GUILTY PLEASURES,2642353.0,Restaurant,Risk 1 (High),4731 W MADISON ST,CHICAGO,IL,60644.0,01/09/2020,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.880401,-87.744090,"(-87.74408951751022, 41.88040118054544)"
106,2352707,SOFI LLC,SOFI LLC,2245297.0,Restaurant,Risk 1 (High),616 S DEARBORN ST,CHICAGO,IL,60605.0,12/04/2019,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.874235,-87.629356,"(-87.62935644330602, 41.87423457984985)"
116,2352266,LOCAL MARKET,LOCAL MARKET,2689709.0,Grocery Store,Risk 1 (High),2101 E 71ST ST,CHICAGO,IL,60649.0,11/25/2019,License,Fail,NaN,41.765990,-87.573787,"(-87.57378733665412, 41.765990001140594)"
129,2345602,VELVET TACO,VELVET TACO,2652941.0,Restaurant,Risk 1 (High),2309 N LINCOLN AVE,CHICAGO,IL,60614.0,11/14/2019,License,Fail,NaN,41.923953,-87.646462,"(-87.6464616293504, 41.9239529807269)"


In [46]:
df.dropna(subset=["Violations"], inplace=True)

In [47]:
df.shape

(37681, 17)

In [48]:
df.head(10)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
33,2360021,I LOVE SUSHI,I LOVE SUSHI,2594986.0,Restaurant,Risk 1 (High),233 N MICHIGAN AVE,CHICAGO,IL,60601.0,02/06/2020,Canvass,Fail,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.886567,-87.624385,"(-87.62438467059714, 41.886567370886944)"
66,2356510,HAILEY'S HOAGIES,HAILEY'S HOAGIES,2583295.0,Restaurant,Risk 1 (High),1055 W 63RD ST,CHICAGO,IL,60621.0,01/10/2020,Canvass,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.779537,-87.651917,"(-87.65191733069446, 41.779536944269374)"
70,2356434,SIMPLY THALIA,SIMPLY THALIA,2059952.0,Restaurant,Risk 1 (High),108 N STATE ST,CHICAGO,IL,60602.0,01/09/2020,Canvass,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.883423,-87.628022,"(-87.62802165207536, 41.88342263701488)"
71,2356436,UNCLE REMUS,UNCLE REMUS PLUS GUILTY PLEASURES,2642353.0,Restaurant,Risk 1 (High),4731 W MADISON ST,CHICAGO,IL,60644.0,01/09/2020,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.880401,-87.744090,"(-87.74408951751022, 41.88040118054544)"
106,2352707,SOFI LLC,SOFI LLC,2245297.0,Restaurant,Risk 1 (High),616 S DEARBORN ST,CHICAGO,IL,60605.0,12/04/2019,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.874235,-87.629356,"(-87.62935644330602, 41.87423457984985)"
130,2345545,THIRD RAIL TAVERN,THIRD RAIL TAVERN,1742274.0,Restaurant,Risk 1 (High),1133 W MADISON ST,CHICAGO,IL,60607.0,11/13/2019,Canvass,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.881534,-87.655650,"(-87.65565021660242, 41.88153435491253)"
137,2345240,MARKET CREATIONS,MARKET CREATIONS,2556028.0,Restaurant,Risk 1 (High),444 W LAKE ST,CHICAGO,IL,60606.0,11/07/2019,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.885830,-87.639806,"(-87.63980643561894, 41.88582970383457)"
154,2320654,CARNICERIA Y TAQUERIA TIERRA,CARNICERIA Y TAQUERIA TIERRA,2428138.0,Grocery Store,Risk 1 (High),3312-3314 W NORTH AVE,CHICAGO,IL,60647.0,10/29/2019,Canvass,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.910185,-87.709907,"(-87.70990705170502, 41.91018535990397)"
169,2320218,CAMERON'S ICE CREAM & GRILL,CAMERON'S ICE CREAM & GRILL,1875095.0,Restaurant,Risk 1 (High),1401 W DIVERSEY PKWY,CHICAGO,IL,60614.0,10/21/2019,Canvass,Fail,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.932370,-87.663625,"(-87.66362490887654, 41.932369724412666)"
170,2316176,DOG HAUS,DOG HAUS,2536489.0,Restaurant,Risk 1 (High),2464-2466 N LINCOLN AVE,CHICAGO,IL,60614.0,10/21/2019,Canvass Re-Inspection,Fail,37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER ...,41.926745,-87.650790,"(-87.65078998020067, 41.92674471870681)"


In [49]:
df_violations = df['Violations']
df_violations.head(10)

33     10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...
66     1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...
70     1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...
71     3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...
106    3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...
130    1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...
137    3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...
154    3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...
169    2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...
170    37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER ...
Name: Violations, dtype: object

In [50]:
descriptions = ""
for i in df_violations.values:
    if not pd.isnull(i): # get rid of the NaNs in this pandas series of violations data
        desc = i.split(' | ') # Reasons for the violations are separated by |
        for text in desc: # for each reason
            if re.match(r'^[0-9]+\.', text): # the reason always starts with numerical digits with a dot
                reason = text.split(' - Comments')[0] # remove the comments part in the cause
                reason = re.sub(r'\n\w+ VIOLATION.*$', " ", reason.strip())
                descriptions+=str(reason)

In [52]:
descriptions[0:1000]


'10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED AND ACCESSIBLE23. PROPER DATE MARKING AND DISPOSITION25. CONSUMER ADVISORY PROVIDED FOR RAW/UNDERCOOKED FOOD37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER37. FOOD PROPERLY LABELED; ORIGINAL CONTAINER39. CONTAMINATION PREVENTED DURING FOOD PREPARATION, STORAGE & DISPLAY41. WIPING CLOTHS: PROPERLY USED & STORED49. NON-FOOD/FOOD CONTACT SURFACES CLEAN49. NON-FOOD/FOOD CONTACT SURFACES CLEAN51. PLUMBING INSTALLED; PROPER BACKFLOW DEVICES55. PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN55. PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN56. ADEQUATE VENTILATION & LIGHTING; DESIGNATED AREAS USED57. ALL FOOD EMPLOYEES HAVE FOOD HANDLER TRAINING58. ALLERGEN TRAINING AS REQUIRED60. PREVIOUS CORE VIOLATION CORRECTED1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOWLEDGE, AND PERFORMS DUTIES2. CITY OF CHICAGO FOOD SERVICE SANITATION CERTIFICATE3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPLOYEE; KNOWLEDGE, RESPONSIBILITIES AND REPORTING5. PROCED

# Step 2: Tokenization of each regulation description

In [55]:
raw_tokens = nltk.word_tokenize(descriptions)


# Step 3: Finding top-10 tokens (for the whole table)

In [56]:
fdist_raw = nltk.FreqDist(raw_tokens)

fdist_raw.most_common(10)

[(',', 321115),
 ('.', 244075),
 ('AND', 159268),
 (':', 119491),
 ('FOOD', 70011),
 ('EQUIPMENT', 63324),
 ('CONSTRUCTED', 60743),
 ('OF', 59363),
 ('PROPERLY', 54091),
 ('INSTALLED', 51141)]

In [57]:
fdist_raw_df = pd.DataFrame(fdist_raw.most_common(),
                    columns=['Word', 'Frequency'])

fdist_raw_df.head(n=10)

,Word,Frequency
0,",",321115
1,.,244075
2,AND,159268
3,:,119491
4,FOOD,70011
5,EQUIPMENT,63324
6,CONSTRUCTED,60743
7,OF,59363
8,PROPERLY,54091
9,INSTALLED,51141


# Step 4: Cleaning the data: converting to lower case, remove stopwords, punctuation, numbers, etc

In [65]:
# Lowercasing all words 
lcase_words = [w.lower() for w in raw_tokens]

#removing stopwords
stop_words = set(nltk.corpus.stopwords.words('english'))

no_stop_words = [word for word in lcase_words if word not in stop_words]

# Removing punctuation or single-character tokens
no_punctuation = [i for i in no_stop_words if len(i) > 1]

# Removing punctuation
clean_data = [word for word in no_punctuation if word.isalpha()]



# Step 5 : finding top 10 token after cleaning the data

In [66]:
fdist_clean = nltk.FreqDist(clean_data)

fdist_clean.most_common(10)

[('food', 70011),
 ('equipment', 63324),
 ('constructed', 60743),
 ('properly', 54091),
 ('installed', 51141),
 ('cleaning', 48295),
 ('clean', 45137),
 ('surfaces', 40535),
 ('maintained', 40347),
 ('contact', 38560)]

In [67]:
fdist_clean_df = pd.DataFrame(fdist_clean.most_common(),
                    columns=['Word', 'Frequency'])

fdist_clean_df.head(n=10)

,Word,Frequency
0,food,70011
1,equipment,63324
2,constructed,60743
3,properly,54091
4,installed,51141
5,cleaning,48295
6,clean,45137
7,surfaces,40535
8,maintained,40347
9,contact,38560


# Step 6:  Top-10 tokens after applying Porter stemming 

In [69]:
porter = nltk.PorterStemmer()

In [70]:
porter_stem_words = [porter.stem(t) for t in clean_data]

In [71]:
fdist_porter = nltk.FreqDist(porter_stem_words)

fdist_porter.most_common(10)

[('clean', 114733),
 ('food', 74434),
 ('equip', 63324),
 ('construct', 60743),
 ('properli', 54091),
 ('instal', 51141),
 ('maintain', 43896),
 ('surfac', 40535),
 ('contact', 38560),
 ('good', 38505)]

In [72]:
fdist_porter_df = pd.DataFrame(fdist_porter.most_common(),
                    columns=['Word', 'Frequency'])

fdist_porter_df.head(n=10)

,Word,Frequency
0,clean,114733
1,food,74434
2,equip,63324
3,construct,60743
4,properli,54091
5,instal,51141
6,maintain,43896
7,surfac,40535
8,contact,38560
9,good,38505


# Step 7 : Top-10 tokens after applying Lancaster stemming

In [76]:
lancaster = nltk.LancasterStemmer()
lancaster_stem_words = [lancaster.stem(t) for t in clean_data]

In [77]:
fdist_lancaster = nltk.FreqDist(lancaster_stem_words)

fdist_lancaster.most_common(10)

[('cle', 118199),
 ('food', 74434),
 ('prop', 65617),
 ('equip', 63324),
 ('construct', 60743),
 ('instal', 51141),
 ('maintain', 43896),
 ('surfac', 40535),
 ('contact', 38560),
 ('good', 38505)]

In [78]:
fdist_lancaster_df = pd.DataFrame(fdist_lancaster.most_common(),
                    columns=['Word', 'Frequency'])

fdist_lancaster_df.head(n=10)

,Word,Frequency
0,cle,118199
1,food,74434
2,prop,65617
3,equip,63324
4,construct,60743
5,instal,51141
6,maintain,43896
7,surfac,40535
8,contact,38560
9,good,38505


# Step 8 : Top-10 tokens after Lemmatization

In [79]:
lemmatizer = nltk.WordNetLemmatizer()


In [80]:
lemmas = [lemmatizer.lemmatize(i.lower()) for i in clean_data]

In [81]:
fdist_lemma = nltk.FreqDist(lemmas)

fdist_lemma.most_common(10)

[('food', 74434),
 ('equipment', 63324),
 ('constructed', 60743),
 ('properly', 54091),
 ('installed', 51141),
 ('cleaning', 48295),
 ('clean', 45137),
 ('surface', 40535),
 ('maintained', 40347),
 ('contact', 38560)]

In [82]:
fdist_lemma_df = pd.DataFrame(fdist_lemma.most_common(),
                    columns=['Word', 'Frequency'])

fdist_lemma_df.head(n=10)

,Word,Frequency
0,food,74434
1,equipment,63324
2,constructed,60743
3,properly,54091
4,installed,51141
5,cleaning,48295
6,clean,45137
7,surface,40535
8,maintained,40347
9,contact,38560


# Step 9 : Comparing top-10 tokens

In [94]:
data = [fdist_raw_df["Word"],fdist_clean_df["Word"],fdist_porter_df["Word"],fdist_lancaster_df["Word"],fdist_lemma_df["Word"] ]
headers = ["fdist_raw_df", "fdist_clean_df","fdist_porter_df","fdist_lancaster_df","fdist_lemma_df"]
compare_df = pd.concat(data, axis=1, keys=headers)

In [96]:
compare_df.head(10)

,fdist_raw_df,fdist_clean_df,fdist_porter_df,fdist_lancaster_df,fdist_lemma_df
0,",",food,clean,cle,food
1,.,equipment,food,food,equipment
2,AND,constructed,equip,prop,constructed
3,:,properly,construct,equip,properly
4,FOOD,installed,properli,construct,installed
5,EQUIPMENT,cleaning,instal,instal,cleaning
6,CONSTRUCTED,clean,maintain,maintain,clean
7,OF,surfaces,surfac,surfac,surface
8,PROPERLY,maintained,contact,contact,maintained
9,INSTALLED,contact,good,good,contact
